In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import folium

In [9]:
ccc = gpd.read_file('dropoutC_boundary_shape/dropoutC_boundary.shp')
ddd = gpd.read_file('dropoutD_boundary_shape/dropoutD_boundary.shp')


In [11]:
income2022 = gpd.read_file('incomesub2022_census.gpkg')
income2023 = gpd.read_file('incomesub2023_census.gpkg')


In [13]:
gdf_merged = gpd.GeoDataFrame(pd.concat([ccc, ddd], axis=0, ignore_index=True))

state_codes = ["04", "08", "12", "13", "20", "22", "36", "41", "42", "48", "51", "55"]
filtered_gdf = gdf_merged[~gdf_merged['STATEFP'].isin(state_codes)]

In [17]:
filtered_gdf = filtered_gdf.to_crs(epsg=2163)
income2022 = income2022.to_crs(epsg=2163)
income2023 = income2023.to_crs(epsg=2163)
filtered_gdf['sd_area'] = filtered_gdf.geometry.area
income2022['div_area'] = income2022.geometry.area
income2023['div_area'] = income2023.geometry.area

In [25]:
intersection2023 = gpd.overlay(filtered_gdf, income2022, how='intersection', keep_geom_type=False)
intersection2023['overlap_area'] = intersection2023.geometry.area
intersection2023['over_pct'] = round((intersection2023['overlap_area']/intersection2023['div_area']),4)
intersection2022 = gpd.overlay(filtered_gdf, income2023, how='intersection', keep_geom_type=False)
intersection2022['overlap_area'] = intersection2022.geometry.area
intersection2022['over_pct'] = round((intersection2022['overlap_area']/intersection2022['div_area']),4)

In [29]:
import shapely
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection

def to_polygon_or_multi(geom):
    if geom.geom_type == 'Polygon' or geom.geom_type == 'MultiPolygon':
        return geom
    
    if geom.geom_type == 'GeometryCollection':
        polys = []
        for g in geom.geoms:
            if g.geom_type == 'Polygon':
                polys.append(g)
            elif g.geom_type == 'MultiPolygon':
                polys.extend(list(g.geoms))
        
        if not polys:
            return None

      
        merged = shapely.ops.unary_union(polys)
        return merged 

    
    return None

In [31]:
intersection2023['geometry'] = intersection2023['geometry'].apply(to_polygon_or_multi)
intersection2022['geometry'] = intersection2022['geometry'].apply(to_polygon_or_multi)

In [27]:
print(intersection2023.geometry.type.value_counts())
print(intersection2022.geometry.type.value_counts())

GeometryCollection    5354
Polygon               2087
MultiPolygon          1040
Point                  326
MultiPoint             172
LineString              64
MultiLineString         12
Name: count, dtype: int64
GeometryCollection    5354
Polygon               2085
MultiPolygon          1042
Point                  326
MultiPoint             170
LineString              64
MultiLineString         12
Name: count, dtype: int64


In [33]:
print(intersection2023.geometry.type.value_counts())
print(intersection2022.geometry.type.value_counts())

MultiPolygon    5540
Polygon         2891
Name: count, dtype: int64
MultiPolygon    5543
Polygon         2889
Name: count, dtype: int64


In [41]:
variables = ['Less_than_10000',
       '10000-14999', '15000-19999', '20000-24999', '25000-29999',
       '30000-34999', '35000-39999', '40000-44999', '45000-49999',
       '50000-59999', '60000-74999', '75000-99999', '100000-124999',
       '125000-149999', '150000-199999', '200000_or_more']
def adj_values(intersection, variables, over='over_pct'):
    for var in variables:
        adj_name = var
        intersection[adj_name] = intersection[over] * intersection[var]
    return intersection

intersection2022 = adj_values(intersection2022, variables)
intersection2023 = adj_values(intersection2023, variables)

In [67]:
constant = ['GEOID_1']
sum_vars = ['overlap_area']
aggregation_dict = {}
aggregation_dict = {var: 'first' for var in constant}
aggregation_dict.update({var: 'sum' for var in variables})
aggregation_dict.update({var: 'sum' for var in sum_vars})


In [109]:
aggregated2023 = intersection2023.dissolve(
    by="GEOID_1", 
    aggfunc=aggregation_dict
).reset_index(drop=True)

aggregated2022 = intersection2022.dissolve(
    by="GEOID_1", 
    aggfunc=aggregation_dict
).reset_index(drop=True)

In [57]:
def calculate_median(row, income_columns, income_ranges):
    cumulative_freq = row[income_columns[0]]
    cumulative_freqs = [cumulative_freq]
    
    for col in income_columns[1:]:
        cumulative_freq += row[col]
        cumulative_freqs.append(cumulative_freq)
    
    T = cumulative_freqs[-1]
    T2 = T / 2

    for i, cumulative_freq in enumerate(cumulative_freqs):
        if cumulative_freq >= T2:
            median_class = income_columns[i]
            lower_bound = income_ranges[i][1]
            upper_bound = income_ranges[i][2]
            CF_m_minus_1 = cumulative_freqs[i-1] if i > 0 else 0
            fm = row[median_class]
            W = upper_bound - lower_bound
            
            if fm == 0:
                return None
            median = lower_bound + (T2 - CF_m_minus_1) / fm * W
            return median
    return None


income_columns = ['Less_than_10000', '10000-14999', '15000-19999', '20000-24999', 
                  '25000-29999', '30000-34999', '35000-39999', '40000-44999', 
                  '45000-49999', '50000-59999', '60000-74999', '75000-99999', 
                  '100000-124999', '125000-149999', '150000-199999', '200000_or_more']

income_ranges = [
    ("Less_than_10000", 0, 10000),
    ("10000-14999", 10000, 14999),
    ("15000-19999", 15000, 19999),
    ("20000-24999", 20000, 24999),
    ("25000-29999", 25000, 29999),
    ("30000-34999", 30000, 34999),
    ("35000-39999", 35000, 39999),
    ("40000-44999", 40000, 44999),
    ("45000-49999", 45000, 49999),
    ("50000-59999", 50000, 59999),
    ("60000-74999", 60000, 74999),
    ("75000-99999", 75000, 99999),
    ("100000-124999", 100000, 124999),
    ("125000-149999", 125000, 149999),
    ("150000-199999", 150000, 199999),
    ("200000_or_more", 200000, float('inf'))
]

In [113]:
aggregated2022['Median_Income'] = aggregated2022.apply(calculate_median, axis=1, income_columns=income_columns, income_ranges=income_ranges)
aggregated2022['Median_Income'] = aggregated2022['Median_Income'].replace(float('inf'), 200000)

aggregated2023['Median_Income'] = aggregated2023.apply(calculate_median, axis=1, income_columns=income_columns, income_ranges=income_ranges)
aggregated2023['Median_Income'] = aggregated2023['Median_Income'].replace(float('inf'), 200000)

In [117]:
income2022 = aggregated2022[['GEOID_1','Median_Income']]
income2023 = aggregated2022[['GEOID_1','Median_Income']]

In [119]:
income2022['Year'] = '2022'
income2023['Year'] = '2023'

/var/folders/9j/4v3vpn4963950lzjkjbvxc740000gn/T/ipykernel_3520/4209202657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income2022['Year'] = '2022'
/var/folders/9j/4v3vpn4963950lzjkjbvxc740000gn/T/ipykernel_3520/4209202657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income2023['Year'] = '2023'


In [123]:
income = pd.concat([income2022, income2023], axis =0)

In [125]:
income.to_csv("income_done.csv", index=False)
